# Este notebook faz parte da reportagem [Censo 2022: a fotografia da população indígena em dados](https://mais.opovo.com.br/reportagens-especiais/2023/08/07/censo-2022-a-fotografia-da-populacao-indigena-por-lentes-full-hd.html?collection=reportagens_de_dados)

Para este material foram consultados dados relacionados à população residente, total e indígena, por localização do domicílio, segundo Unidades da Federação, Terras Indígenas e Municípios Cearenses (Censo Demográfico 2022) do Instituto Brasileiro de Geografia e Estatística - IBGE.

In [1]:
import pandas as pd
import geopandas as gpd
import os
from shapely import wkt
from shapely.geometry import Point, MultiPolygon

In [2]:
cidades_ibge_geometrias = pd.read_csv('bases_de_dados/cidades_ibge_geometrias.csv')

In [3]:
cidades_ibge_geometrias = cidades_ibge_geometrias[['cod_ibge7', 'municipio', 'cod_uf', 'Nome_UF',
       'UF', 'Nome_Município', 'uf_min', 'geometry']]

In [4]:
geometrias_cidades_ce = cidades_ibge_geometrias.loc[cidades_ibge_geometrias.uf_min=='ce']
geometrias_cidades_ce

,cod_ibge7,municipio,cod_uf,Nome_UF,UF,Nome_Município,uf_min,geometry
619,2300101,ABAIARA,23,Ceará,CE,Abaiara,ce,"POLYGON ((-38.9699 -7.4273, -38.969 -7.4248, -..."
620,2300150,ACARAPE,23,Ceará,CE,Acarape,ce,"POLYGON ((-38.6188 -4.1635, -38.6199 -4.1638, ..."
621,2300200,ACARAÚ,23,Ceará,CE,Acaraú,ce,"POLYGON ((-39.9424 -2.8664, -39.9444 -2.8646, ..."
622,2300309,ACOPIARA,23,Ceará,CE,Acopiara,ce,"POLYGON ((-39.0507 -6.0254, -39.053 -6.0181, -..."
623,2300408,AIUABA,23,Ceará,CE,Aiuaba,ce,"POLYGON ((-40.0841 -6.3674, -40.0848 -6.3665, ..."
...,...,...,...,...,...,...,...,...
798,2313807,URUBURETAMA,23,Ceará,CE,Uruburetama,ce,"POLYGON ((-39.4556 -3.6431, -39.4205 -3.6262, ..."
799,2313906,URUOCA,23,Ceará,CE,Uruoca,ce,"POLYGON ((-40.7279 -3.2339, -40.7944 -3.218, -..."
800,2313955,VARJOTA,23,Ceará,CE,Varjota,ce,"POLYGON ((-40.451 -4.2091, -40.45 -4.2076, -40..."
801,2314102,VIÇOSA DO CEARÁ,23,Ceará,CE,Viçosa do Ceará,ce,"POLYGON ((-40.8867 -3.5204, -40.9298 -3.4845, ..."


In [5]:
path_tabela9718 = 'bases_de_dados/tabela9718_tratada.csv'

In [6]:
tabela9718 = pd.read_csv(path_tabela9718)

In [7]:
tabela9718.sum()

cidade                      AbaiaraAcarapeAcaraúAcopiaraAiuabaAlcântarasAl...
Total_total                                                             56353
Total_cor_ou_raca                                                       39980
Total_se_considera                                                      16373
Total_nao_se_declara                                                        0
EmTI_total                                                              10524
EmTI_cor_ou_raca                                                         9275
EmTI_se_considera                                                        1249
EmTI_nao_se_declara                                                         0
Pop_total                                                             8791688
Pop_cor_ou_raca                                                         39980
Pop_se_considera                                                        16373
Pop_nao_se_declara                                              

In [8]:
tabela9718['CIDADE']=tabela9718.cidade.apply(lambda x: x.upper())

In [9]:
tabela9718.head()

,cidade,Total_total,Total_cor_ou_raca,Total_se_considera,Total_nao_se_declara,EmTI_total,EmTI_cor_ou_raca,EmTI_se_considera,EmTI_nao_se_declara,Pop_total,Pop_cor_ou_raca,Pop_se_considera,Pop_nao_se_declara,Pop_em_TI_total,Pop_em_TI_cor_ou_raca,Pop_em_TI_se_considera,Pop_em_TI_nao_se_declara,CIDADE
0,Abaiara,5,5,0,0,0,0,0,0,10038,5,0,10033,0,0,0,0,ABAIARA
1,Acarape,7,7,0,0,0,0,0,0,14027,7,0,14020,0,0,0,0,ACARAPE
2,Acaraú,657,500,157,0,401,373,28,0,64806,500,157,64149,422,373,28,21,ACARAÚ
3,Acopiara,24,24,0,0,0,0,0,0,44962,24,0,44938,0,0,0,0,ACOPIARA
4,Aiuaba,4,4,0,0,0,0,0,0,14076,4,0,14072,0,0,0,0,AIUABA


In [10]:
geometrias_cidades_ce.head()

,cod_ibge7,municipio,cod_uf,Nome_UF,UF,Nome_Município,uf_min,geometry
619,2300101,ABAIARA,23,Ceará,CE,Abaiara,ce,"POLYGON ((-38.9699 -7.4273, -38.969 -7.4248, -..."
620,2300150,ACARAPE,23,Ceará,CE,Acarape,ce,"POLYGON ((-38.6188 -4.1635, -38.6199 -4.1638, ..."
621,2300200,ACARAÚ,23,Ceará,CE,Acaraú,ce,"POLYGON ((-39.9424 -2.8664, -39.9444 -2.8646, ..."
622,2300309,ACOPIARA,23,Ceará,CE,Acopiara,ce,"POLYGON ((-39.0507 -6.0254, -39.053 -6.0181, -..."
623,2300408,AIUABA,23,Ceará,CE,Aiuaba,ce,"POLYGON ((-40.0841 -6.3674, -40.0848 -6.3665, ..."


In [11]:
dataset_pop_idg = pd.merge(geometrias_cidades_ce, tabela9718, left_on='municipio', right_on='CIDADE')

In [12]:
dataset_pop_idg.columns

Index(['cod_ibge7', 'municipio', 'cod_uf', 'Nome_UF', 'UF', 'Nome_Município',
       'uf_min', 'geometry', 'cidade', 'Total_total', 'Total_cor_ou_raca',
       'Total_se_considera', 'Total_nao_se_declara', 'EmTI_total',
       'EmTI_cor_ou_raca', 'EmTI_se_considera', 'EmTI_nao_se_declara',
       'Pop_total', 'Pop_cor_ou_raca', 'Pop_se_considera',
       'Pop_nao_se_declara', 'Pop_em_TI_total', 'Pop_em_TI_cor_ou_raca',
       'Pop_em_TI_se_considera', 'Pop_em_TI_nao_se_declara', 'CIDADE'],
      dtype='object')

In [13]:
dataset_pop_idg['pct_IDG-Total'] = dataset_pop_idg.apply(lambda x: round(x.Total_total/x.Pop_total*100,2),axis=1)

dataset_pop_idg['pct_emTI'] = dataset_pop_idg.apply(lambda x: round(x.EmTI_total/x.Total_total*100,2) if x.Total_total >0 else 0,axis=1)


dataset_pop_idg['pct_foraTI'] = dataset_pop_idg.apply(lambda x: 100-round(x.EmTI_total/x.Total_total*100,2) if x.Total_total >0 else 0,axis=1)

In [14]:
dataset_pop_idg['pct_foraTI'].describe()

count    184.000000
mean      91.735163
std       26.131431
min        0.000000
25%      100.000000
50%      100.000000
75%      100.000000
max      100.000000
Name: pct_foraTI, dtype: float64

In [15]:
dataset_pop_idg['pct_IDG-Total'].describe()

count    184.000000
mean       0.573098
std        2.574172
min        0.000000
25%        0.030000
50%        0.060000
75%        0.132500
max       28.350000
Name: pct_IDG-Total, dtype: float64

In [16]:
dataset_pop_idg['Total_total'].describe()

count      184.000000
mean       306.266304
std       1510.531050
min          0.000000
25%          5.000000
50%         10.500000
75%         55.250000
max      17628.000000
Name: Total_total, dtype: float64

In [17]:
dataset_pop_idg[['cidade','Total_total','pct_IDG-Total']].head(5)

,cidade,Total_total,pct_IDG-Total
0,Abaiara,5,0.05
1,Acarape,7,0.05
2,Acaraú,657,1.01
3,Acopiara,24,0.05
4,Aiuaba,4,0.03


In [18]:
dataset_pop_idg.sort_values(by='Total_total', ascending=False).head(5)[['municipio','Total_total','pct_IDG-Total','EmTI_total']]

,municipio,Total_total,pct_IDG-Total,EmTI_total
43,CAUCAIA,17628,4.96,5791
89,ITAREMA,5115,11.97,508
103,MARACANAÚ,5111,2.18,2437
58,FORTALEZA,4948,0.20,0
115,MONSENHOR TABOSA,4861,28.35,0


In [19]:
dataset_pop_idg.sort_values(by='pct_emTI', ascending=False).head(10)[['municipio','pct_emTI','Total_total','pct_IDG-Total']]

,municipio,pct_emTI,Total_total,pct_IDG-Total
87,ITAPIPOCA,72.75,745,0.57
11,AQUIRAZ,62.50,544,0.68
2,ACARAÚ,61.04,657,1.01
103,MARACANAÚ,47.68,5111,2.18
127,PACATUBA,33.98,1486,1.83
43,CAUCAIA,32.85,17628,4.96
89,ITAREMA,9.93,5115,11.97
0,ABAIARA,0.00,5,0.05
128,PACOTI,0.00,5,0.04
120,MULUNGU,0.00,11,0.10


In [20]:
dataset_pop_idg.sort_values(by='pct_IDG-Total', ascending=False).head(5)[['municipio','Total_total','pct_IDG-Total']]

,municipio,Total_total,pct_IDG-Total
115,MONSENHOR TABOSA,4861,28.35
143,PORANGA,1542,12.78
89,ITAREMA,5115,11.97
16,ARATUBA,814,7.25
39,CARNAUBAL,866,5.03


In [21]:
dataset_pop_idg[['cod_ibge7','cidade', 'pct_IDG-Total','Total_total', 'Total_cor_ou_raca',
       'Total_se_considera', 'Total_nao_se_declara', 'EmTI_total','pct_emTI','pct_foraTI',
       'EmTI_cor_ou_raca', 'EmTI_se_considera', 'EmTI_nao_se_declara',
       'Pop_total', 'Pop_cor_ou_raca', 'Pop_se_considera',
       'Pop_nao_se_declara', 'Pop_em_TI_total', 'Pop_em_TI_cor_ou_raca',
       'Pop_em_TI_se_considera', 'Pop_em_TI_nao_se_declara']].to_csv('resultados/dataset_popIDG_mun.csv', index=False)

In [22]:
dataset_pop_idg.head()

,cod_ibge7,municipio,cod_uf,Nome_UF,UF,Nome_Município,uf_min,geometry,cidade,Total_total,...,Pop_se_considera,Pop_nao_se_declara,Pop_em_TI_total,Pop_em_TI_cor_ou_raca,Pop_em_TI_se_considera,Pop_em_TI_nao_se_declara,CIDADE,pct_IDG-Total,pct_emTI,pct_foraTI
0,2300101,ABAIARA,23,Ceará,CE,Abaiara,ce,"POLYGON ((-38.9699 -7.4273, -38.969 -7.4248, -...",Abaiara,5,...,0,10033,0,0,0,0,ABAIARA,0.05,0.00,100.00
1,2300150,ACARAPE,23,Ceará,CE,Acarape,ce,"POLYGON ((-38.6188 -4.1635, -38.6199 -4.1638, ...",Acarape,7,...,0,14020,0,0,0,0,ACARAPE,0.05,0.00,100.00
2,2300200,ACARAÚ,23,Ceará,CE,Acaraú,ce,"POLYGON ((-39.9424 -2.8664, -39.9444 -2.8646, ...",Acaraú,657,...,157,64149,422,373,28,21,ACARAÚ,1.01,61.04,38.96
3,2300309,ACOPIARA,23,Ceará,CE,Acopiara,ce,"POLYGON ((-39.0507 -6.0254, -39.053 -6.0181, -...",Acopiara,24,...,0,44938,0,0,0,0,ACOPIARA,0.05,0.00,100.00
4,2300408,AIUABA,23,Ceará,CE,Aiuaba,ce,"POLYGON ((-40.0841 -6.3674, -40.0848 -6.3665, ...",Aiuaba,4,...,0,14072,0,0,0,0,AIUABA,0.03,0.00,100.00


In [23]:
# Região Metropolitana do Cariri:

In [24]:
crajubar = dataset_pop_idg.loc[dataset_pop_idg.Nome_Município.isin(['Barbalha','Caririaçu','Crato','Farias Brito','Jardim','Juazeiro do Norte','Missão Velha','Nova Olinda','Santana do Cariri'])]

In [25]:
crajubar[['cidade', 'pct_IDG-Total','Total_total', 'Total_cor_ou_raca',
       'Total_se_considera', 'Total_nao_se_declara', 'EmTI_total','pct_emTI','pct_foraTI',
       'EmTI_cor_ou_raca', 'EmTI_se_considera', 'EmTI_nao_se_declara',
       'Pop_total', 'Pop_cor_ou_raca', 'Pop_se_considera',
       'Pop_nao_se_declara', 'Pop_em_TI_total', 'Pop_em_TI_cor_ou_raca',
       'Pop_em_TI_se_considera', 'Pop_em_TI_nao_se_declara']]

,cidade,pct_IDG-Total,Total_total,Total_cor_ou_raca,Total_se_considera,Total_nao_se_declara,EmTI_total,pct_emTI,pct_foraTI,EmTI_cor_ou_raca,EmTI_se_considera,EmTI_nao_se_declara,Pop_total,Pop_cor_ou_raca,Pop_se_considera,Pop_nao_se_declara,Pop_em_TI_total,Pop_em_TI_cor_ou_raca,Pop_em_TI_se_considera,Pop_em_TI_nao_se_declara
22,Barbalha,0.10,74,74,0,0,0,0.0,100.0,0,0,0,75033,74,0,74959,0,0,0,0
36,Caririaçu,0.08,21,17,4,0,0,0.0,100.0,0,0,0,26320,17,4,26299,0,0,0,0
50,Crato,0.27,353,353,0,0,0,0.0,100.0,0,0,0,131050,353,0,130697,0,0,0,0
56,Farias Brito,0.04,8,8,0,0,0,0.0,100.0,0,0,0,18217,8,0,18209,0,0,0,0
95,Jardim,0.44,121,31,90,0,0,0.0,100.0,0,0,0,27411,31,90,27290,0,0,0,0
98,Juazeiro do Norte,0.12,351,248,103,0,0,0.0,100.0,0,0,0,286120,248,103,285769,0,0,0,0
113,Missão Velha,0.15,55,44,11,0,0,0.0,100.0,0,0,0,36822,44,11,36767,0,0,0,0
121,Nova Olinda,0.03,5,5,0,0,0,0.0,100.0,0,0,0,15399,5,0,15394,0,0,0,0
159,Santana do Cariri,0.03,5,5,0,0,0,0.0,100.0,0,0,0,16954,5,0,16949,0,0,0,0


In [26]:
crajubar[['cidade', 'Total_total', 'Total_cor_ou_raca',
       'Total_se_considera', 'Total_nao_se_declara', 'EmTI_total','pct_emTI','pct_foraTI',
       'EmTI_cor_ou_raca', 'EmTI_se_considera', 'EmTI_nao_se_declara',
       'Pop_total', 'Pop_cor_ou_raca', 'Pop_se_considera',
       'Pop_nao_se_declara', 'Pop_em_TI_total', 'Pop_em_TI_cor_ou_raca',
       'Pop_em_TI_se_considera', 'Pop_em_TI_nao_se_declara']].sum()

cidade                      BarbalhaCaririaçuCratoFarias BritoJardimJuazei...
Total_total                                                               993
Total_cor_ou_raca                                                         785
Total_se_considera                                                        208
Total_nao_se_declara                                                        0
EmTI_total                                                                  0
pct_emTI                                                                  0.0
pct_foraTI                                                              900.0
EmTI_cor_ou_raca                                                            0
EmTI_se_considera                                                           0
EmTI_nao_se_declara                                                         0
Pop_total                                                              633326
Pop_cor_ou_raca                                                 

In [27]:
# População por TI

In [28]:
path_pop_ti = 'bases_de_dados/pop_ti_2010_2022.csv'

In [29]:
pop_idg = pd.read_csv(path_pop_ti)

In [30]:
pop_idg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573 entries, 0 to 572
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   cod_ti        573 non-null    int64  
 1   nome_ti       573 non-null    object 
 2   pop_res_2010  501 non-null    float64
 3   pop_idg_2010  478 non-null    float64
 4   pop_res_2022  560 non-null    float64
 5   pop_idg_2022  533 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 27.0+ KB


In [31]:
pop_idg.sum()

cod_ti                                                   14809117
nome_ti         Acapuri de CimaAcimãAconãÁgua Preta/InariÁguas...
pop_res_2010                                             561402.0
pop_idg_2010                                             511050.0
pop_res_2022                                             689202.0
pop_idg_2022                                             620855.0
dtype: object

In [32]:
gdf_ti = gpd.read_file('bases_de_dados/br_ti.geojson')

/home/alexandre/anaconda3/envs/credos/lib/python3.7/site-packages/geopandas/array.py:93: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  aout[:] = out


In [33]:
gdf_ti['cod_ti'] = gdf_ti.COD_TI.astype(int)

In [34]:
pop_idg_ti = gdf_ti.merge(pop_idg, on='cod_ti')
pop_idg_ti.head()

,COD_TI,NOM_TI,COD_FUNAI,NOM_FUNAI,FASE,geometry,cod_ti,nome_ti,pop_res_2010,pop_idg_2010,pop_res_2022,pop_idg_2022
0,10430,Ribeirão Silveira,38102,Ribeirão Silveira,Declarada,"MULTIPOLYGON (((-45.81322 -23.74269, -45.81314...",10430,Ribeirão Silveira,2892.0,332.0,296.0,152.0
1,10448,Boa Vista Sertão do Promirim,6301,Boa Vista Sertão do Promirim,Regularizada,"MULTIPOLYGON (((-45.00638 -23.33642, -45.00589...",10448,Boa Vista Sertão do Promirim,156.0,156.0,195.0,193.0
2,10472,Rio Branco Itanhaém,38701,Rio Branco Itanhaém,Regularizada,"MULTIPOLYGON (((-46.62015 -24.00783, -46.62036...",10472,Rio Branco Itanhaém,80.0,80.0,30.0,30.0
3,10480,Jaraguá,18801,Jaraguá,Regularizada,"MULTIPOLYGON (((-46.75465 -23.46538, -46.75469...",10480,Jaraguá,98.0,98.0,92.0,78.0
4,10510,Krukutu,23601,Krukutu,Regularizada,"MULTIPOLYGON (((-46.61775 -23.86435, -46.61778...",10510,Krukutu,254.0,254.0,155.0,155.0


In [35]:
pop_idg_ti = pop_idg_ti[['cod_ti', 'nome_ti', 'FASE', 'pop_res_2010', 'pop_idg_2010', 'pop_res_2022',
       'pop_idg_2022', 'geometry', 'COD_FUNAI', 'NOM_FUNAI']]

In [36]:
pop_idg_ti['crescimento_2010_2022'] = pop_idg_ti.apply(lambda x: x.pop_idg_2022-x.pop_idg_2010, axis=1)

In [37]:
pop_idg_ti['var_pct'] = pop_idg_ti.apply(lambda x: round((x.pop_idg_2022/x.pop_idg_2010-1)*100,2) if x.pop_idg_2010>0 else None, axis=1)

In [38]:
pop_idg_ti.head(2)

,cod_ti,nome_ti,FASE,pop_res_2010,pop_idg_2010,pop_res_2022,pop_idg_2022,geometry,COD_FUNAI,NOM_FUNAI,crescimento_2010_2022,var_pct
0,10430,Ribeirão Silveira,Declarada,2892.0,332.0,296.0,152.0,"MULTIPOLYGON (((-45.81322 -23.74269, -45.81314...",38102,Ribeirão Silveira,-180.0,-54.22
1,10448,Boa Vista Sertão do Promirim,Regularizada,156.0,156.0,195.0,193.0,"MULTIPOLYGON (((-45.00638 -23.33642, -45.00589...",6301,Boa Vista Sertão do Promirim,37.0,23.72


In [39]:
pop_idg_ti.sort_values(by='pop_idg_2022',ascending=False).head(10)[['nome_ti','FASE','pop_idg_2022','pop_idg_2010','crescimento_2010_2022','var_pct']]

,nome_ti,FASE,pop_idg_2022,pop_idg_2010,crescimento_2010_2022,var_pct
507,Yanomami,Regularizada,27152.0,25719.0,1433.0,5.57
552,Raposa Serra do Sol,Regularizada,26176.0,17102.0,9074.0,53.06
444,Évare I,Regularizada,20177.0,16686.0,3491.0,20.92
68,Alto Rio Negro,Regularizada,18042.0,15183.0,2859.0,18.83
496,Andirá-Marau,Regularizada,14307.0,11321.0,2986.0,26.38
401,Dourados,Regularizada,13473.0,11138.0,2335.0,20.96
383,Potiguara,Regularizada,10960.0,8715.0,2245.0,25.76
44,Cana Brava/Guajajara,Regularizada,10662.0,7133.0,3529.0,49.47
550,São Marcos - RR,Regularizada,10328.0,4945.0,5383.0,108.86
442,Arariboia,Regularizada,10158.0,6997.0,3161.0,45.18


In [40]:
resumo_fase = pop_idg_ti.groupby('FASE')['pop_idg_2022'].sum().reset_index()

total = resumo_fase.pop_idg_2022.sum()

resumo_fase['pct'] = round(resumo_fase.pop_idg_2022/total*100,2)
resumo_fase

,FASE,pop_idg_2022,pct
0,Declarada,39848.0,6.42
1,Encaminhada RI,3877.0,0.62
2,Homologada,3105.0,0.50
3,Regularizada,574025.0,92.46


In [41]:
pop_idg_ti.to_file('resultados/pop_idg_ti.geojson', drive='GeoJson')

In [42]:
pop_idg_ti['lng'] = pop_idg_ti.geometry.apply(lambda x: x.centroid.x)
pop_idg_ti['lat'] = pop_idg_ti.geometry.apply(lambda x: x.centroid.y)

In [43]:
pop_idg_ti['pop_idg_2022'] = round(pop_idg_ti['pop_idg_2022'])
pop_idg_ti['pop_idg_2010'] = round(pop_idg_ti['pop_idg_2010'])

In [44]:
pop_idg_ti[['lat','lng','nome_ti','FASE','pop_idg_2022','pop_idg_2010','crescimento_2010_2022','var_pct']].to_csv('resultados/loc_ti.csv', index=False)

In [45]:
# pop_idg_ti.loc[pop_idg_ti.pop_idg_2022.isna(),'pop_idg_2022'] = 'X'

In [46]:
import math
pop_idg_ti.pop_idg_2010 = pop_idg_ti.pop_idg_2010.apply(lambda x: int(x) if not math.isnan(x) else 'X')

In [47]:
pop_idg_ti

,cod_ti,nome_ti,FASE,pop_res_2010,pop_idg_2010,pop_res_2022,pop_idg_2022,geometry,COD_FUNAI,NOM_FUNAI,crescimento_2010_2022,var_pct,lng,lat
0,10430,Ribeirão Silveira,Declarada,2892.0,332,296.0,152.0,"MULTIPOLYGON (((-45.81322 -23.74269, -45.81314...",38102,Ribeirão Silveira,-180.0,-54.22,-45.805988,-23.710900
1,10448,Boa Vista Sertão do Promirim,Regularizada,156.0,156,195.0,193.0,"MULTIPOLYGON (((-45.00638 -23.33642, -45.00589...",6301,Boa Vista Sertão do Promirim,37.0,23.72,-44.981999,-23.353579
2,10472,Rio Branco Itanhaém,Regularizada,80.0,80,30.0,30.0,"MULTIPOLYGON (((-46.62015 -24.00783, -46.62036...",38701,Rio Branco Itanhaém,-50.0,-62.50,-46.663593,-23.999758
3,10480,Jaraguá,Regularizada,98.0,98,92.0,78.0,"MULTIPOLYGON (((-46.75465 -23.46538, -46.75469...",18801,Jaraguá,-20.0,-20.41,-46.754245,-23.464253
4,10510,Krukutu,Regularizada,254.0,254,155.0,155.0,"MULTIPOLYGON (((-46.61775 -23.86435, -46.61778...",23601,Krukutu,-99.0,-38.98,-46.621036,-23.863625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,9954,Mãe Maria,Regularizada,918.0,896,1302.0,1110.0,"MULTIPOLYGON (((-48.99077 -4.97749, -48.99040 ...",26001,Mãe Maria,214.0,23.88,-48.918534,-5.119119
569,9962,Sororó,Regularizada,343.0,338,594.0,575.0,"MULTIPOLYGON (((-48.59738 -5.97701, -48.59609 ...",43201,Sororó,237.0,70.12,-48.692717,-5.936776
570,9970,Xikrin do Rio Catete,Regularizada,1059.0,1059,1737.0,1727.0,"MULTIPOLYGON (((-50.72127 -5.95701, -50.72126 ...",50501,Xikrin do Rio Catete,668.0,63.08,-50.814802,-6.357282
571,9989,Nhamundá/Mapuera,Regularizada,1965.0,1961,2009.0,2004.0,"MULTIPOLYGON (((-57.99233 -0.10302, -57.99200 ...",30501,Nhamundá/Mapuera,43.0,2.19,-58.110754,-0.821397
